This is one of the Objectiv [example notebooks](https://objectiv.io/docs/modeling/example-notebooks/). These notebooks can also run [on your own data](https://objectiv.io/docs/modeling/get-started-in-your-notebook/) (see [how to set up tracking](https://objectiv.io/docs/tracking/)).

# Feature engineering

This example shows how modeling library Bach can be used for feature engineering. We'll go through describing the data, finding outliers, transforming data, and grouping & aggregating data so that a useful feature set is created that can be used for machine learning. To see details of how such a dataset can be used for machine learning with sklearn, [see our ML notebook](https://objectiv.io/docs/modeling/example-notebooks/machine-learning/).

## Get started
We first have to instantiate the model hub and an Objectiv DataFrame object.

In [ ]:
# set the timeframe of the analysis
start_date = '2022-03-01'
end_date = None

In [ ]:
from modelhub import ModelHub, display_sql_as_markdown

# instantiate the model hub and set the default time aggregation to daily
# and set the global contexts that will be used in this example
modelhub = ModelHub(time_aggregation='%Y-%m-%d', global_contexts=['application'])
# get a Bach DataFrame with Objectiv data within a defined timeframe
df = modelhub.get_objectiv_dataframe(start_date=start_date, end_date=end_date)

This object points to all data in the dataset, which is too large to run in pandas and therefore sklearn. For 
the dataset that we need, we will aggregate to user level, at which point it is small enough to fit in memory.

We'll start with showing the first couple of rows and describing the entire dataset.

### Reference
* [modelhub.ModelHub](https://objectiv.io/docs/modeling/open-model-hub/api-reference/ModelHub/ModelHub/)
* [modelhub.ModelHub.get_objectiv_dataframe](https://objectiv.io/docs/modeling/open-model-hub/api-reference/ModelHub/get_objectiv_dataframe/)

## Describe the data

We start with showing the first couple of rows from the data set and describing the entire data set.

In [ ]:
df.head()

Columns of interest are `user_id`, as this is what we will aggregate to, and `moment`, as this contains timestamp info for the
events. 

See the [open taxonomy example](open-taxonomy-how-to.ipynb#Location-stack-&-global-contexts) for how to use the `location_stack` and `global_contexts` columns.

Now let's look some more into our dataset to see what it contains.

In [ ]:
df.describe(include='all').head()

### Reference
* [bach.DataFrame.describe](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/describe/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)

## Create a feature set 
We'd like to create a feature set that describes the behavior of users in a certain way. We start with extracting the [`root_location`](https://objectiv.io/docs/taxonomy/reference/location-contexts/RootLocationContext) from the location stack, which captures the main areas users have visited. Using `to_numpy()` shows the results as a numpy array.

In [ ]:
df['root_location'] = df.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')

# root series is later unstacked and its values might contain dashes
# which are not allowed in BigQuery column names, lets replace them
df['root_location'] = df['root_location'].str.replace('-', '_')
df.root_location.unique().to_numpy()

This returns `['jobs', 'docs', 'home'...]` etc., which in this example are the sections of the objectiv.io website.

### Check any missing values

In [ ]:
df.root_location.isnull().value_counts().head()

This shows us that there are no missing values to worry about. Now we want a dataset with interactions on our different sections. In particular, [PressEvents](https://objectiv.io/docs/taxonomy/reference/events/PressEvent), an event type. We first want an overview of the different event types that exist and select the one we are interested in.

In [ ]:
df.event_type.unique().to_numpy()

We are interested in all `PressEvent` event types:

In [ ]:
df[(df.event_type=='PressEvent')].root_location.unique().to_numpy()

In [ ]:
df[(df.event_type=='PressEvent')].describe(include='string').head()

### Create the variables
Here we select only PressEvents and then group by `user_id` & `root`, and count the session_hit_number. After that the results are unstacked, resulting in a table where each row represents a user (the index is `user_id`), the columns are the different root_locations, and its values are the number of times a user clicked in those sections.

In [ ]:
features = df[(df.event_type=='PressEvent')].groupby(['user_id','root_location']).session_hit_number.count()

In [ ]:
features_unstacked = features.unstack()

In [ ]:
features_unstacked.materialize().describe().head()

In [ ]:
features_unstacked.head()

### Fill empty values
Now we do have empty values, so we fill them with 0, as empty means that the user did not click in the
section.

In [ ]:
features_unstacked = features.unstack(fill_value=0)

### Describe the dataset again
We use describe again to get an impression of out created per-user data set.

In [ ]:
features_unstacked.materialize().describe().head()

Looking at the mean, some `root_locations` seem to be used a lot more than others. Also, the max number of clicks seems quite different per `root_location`. This information can be used to drop some of the variables from our dataset, or to use scaling or outlier detection. We will plot histograms for this.

### Visualize the data

In [ ]:
from matplotlib import pyplot as plt
import math

figure, axis = plt.subplots(math.ceil(len(features_unstacked.data_columns)/4), 4, figsize=(15,10))

for idx, name in enumerate(features_unstacked.data_columns):
    features_unstacked[[name]].plot.hist(bins=5, title=name, ax=axis.flat[idx])
plt.tight_layout()

The histograms show that indeed the higher values seem quite anomalous for most of the root_locations. This could be a reason to drop some of these observations, or resort to scaling methods. For now we continue with the dataset as is.

### Add time feature
Now we want to add the time feature to our dataset. We'll add the average session length per user for that, using `fillna`to fill missing values.

In [ ]:
import datetime

features_unstacked['session_duration'] = modelhub.aggregate.session_duration(df, groupby='user_id')
features_unstacked['session_duration'] = features_unstacked['session_duration'].fillna(datetime.timedelta(0))

In [ ]:
features_unstacked.session_duration.describe().head()

### Reference
* [using global context data](open-taxonomy-how-to.ipynb#Location-stack-&-global-contexts)
* [modelhub.SeriesLocationStack.ls](https://objectiv.io/docs/modeling/open-model-hub/api-reference/SeriesLocationStack/ls/)
* [bach.Series.unique](https://objectiv.io/docs/modeling/bach/api-reference/Series/unique/)
* [bach.Series.to_numpy](https://objectiv.io/docs/modeling/bach/api-reference/Series/to_numpy/)
* [bach.DataFrame.describe](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/describe/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)
* [bach.Series.count](https://objectiv.io/docs/modeling/bach/api-reference/Series/count/)
* [bach.DataFrame.count](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/count/)
* [bach.DataFrame.materialize](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/materialize/)
* [bach.DataFrame.unstack](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/unstack/)
* [modelhub.Aggregate.session_duration](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/session_duration/)
* [bach.DataFrame.fillna](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/fillna/)

## Export to pandas for sklearn
Now that we have our dataset, we can use it for machine learning, e.g. with sklearn. To do so, we call `to_pandas()` to get a pandas DataFrame that can be used in sklearn.

Also [see the example notebook on how to use Objectiv data and sklearn](https://objectiv.io/docs/modeling/example-notebooks/machine-learning/).

In [ ]:
pdf = features_unstacked.to_pandas()
pdf

### Reference
* [bach.DataFrame.to_pandas](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/to_pandas/)

## Get the SQL for any analysis
The SQL for any analysis can be exported with one command, so you can use models in production directly to simplify data debugging & delivery to BI tools like Metabase, dbt, etc. See how you can [quickly create BI dashboards with this](https://objectiv.io/docs/home/up#creating-bi-dashboards).

In [ ]:
# show SQL for analysis; this is just one example, and works for any Objectiv model/analysis
# complex SQL statement alert!
display_sql_as_markdown(features_unstacked)

That's it! [Join us on Slack](https://objectiv.io/join-slack) if you have any questions or suggestions.

# Next Steps

### Use this notebook with your own data

You can use the example notebooks on any dataset that was collected with Objectiv's tracker, so feel free to 
use them to bootstrap your own projects. They are available as Jupyter notebooks on our [GitHub repository](https://github.com/objectiv/objectiv-analytics/tree/main/notebooks). See [instructions to set up the Objectiv tracker](https://objectiv.io/docs/tracking/).

### Check out related example notebooks

* [Bach and sklearn](./sklearn-example.ipynb) - see how you can do all your analysis and Machine Learning directly on the raw data in your SQL database with Objectiv.